<a href="https://colab.research.google.com/github/yangxuan8/EVSIvsLSTM/blob/main/greedy_cmi_estimation_pl123.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive')

Mounted at /content/drive


In [2]:
!pip install pytorch_lightning==1.9.4
import torch
import pickle
import argparse
import numpy as np
import torch.nn as nn
from torchmetrics import AUROC
from torch.utils.data import DataLoader, random_split
import pandas as pd
sys.path.append('/content/drive/MyDrive/DIME_main/experiments/hev')
import feature_groups
sys.path.append('/content/drive/MyDrive/DIME_main')
from dime.data_utils import HEVDataset, get_group_matrix, get_xy
from dime.utils import MaskLayerGrouped
from dime import MaskingPretrainer, CMIEstimator
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
import time

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 827.8/827.8 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 18.9 MB/s eta 0:00:00


In [3]:
# Set up command line arguments
parser = argparse.ArgumentParser()
parser.add_argument('--gpu', type=int, default=0)
parser.add_argument('--use_feature_costs', default=False, action="store_true")
parser.add_argument('--num_trials', type=int, default=5)

_StoreAction(option_strings=['--num_trials'], dest='num_trials', nargs=None, const=None, default=5, type=<class 'int'>, choices=None, required=False, help=None, metavar=None)

In [5]:
if __name__ == '__main__':
    hev_feature_names = feature_groups.hev_feature_names
    hev_feature_groups = feature_groups.hev_feature_groups

    # Parse args
    args = parser.parse_known_args()[0]
    cols_to_drop = []
    if cols_to_drop is not None:
        hev_feature_names = [item for item in hev_feature_names if str(hev_feature_names.index(item))
                               not in cols_to_drop]

    # Load dataset
    dataset = HEVDataset('data/hev.csv', cols_to_drop=cols_to_drop)
    d_in = dataset.X.shape[1]  # 32
    d_out = len(np.unique(dataset.Y))  # 3

    # Get features and groups
    feature_groups_dict, feature_groups_mask = get_group_matrix(hev_feature_names, hev_feature_groups)
    num_groups = len(feature_groups_mask)  # 45
    print("Num groups=", num_groups)
    print("Num features=", d_in)

    print(feature_groups_dict.keys())

    # Split dataset
    train_dataset, val_dataset, test_dataset = random_split(dataset, [0.8, 0.1, 0.1], generator=torch.Generator().manual_seed(0))

    daataset_dict = dict(train_dataset=train_dataset, val_dataset=val_dataset, test_dataset=test_dataset)
    f = open('/content/drive/MyDrive/dataset/data/dataset11.pkl', "wb", pickle.HIGHEST_PROTOCOL)
    pickle.dump(daataset_dict, f)

    # train_dataset.X
    print(f'Train samples = {len(train_dataset)}, val samples = {len(val_dataset)}, test samples = {len(test_dataset)}')

    # Find mean/variance for normalizing
    x, y = get_xy(train_dataset)
    mean = np.mean(x, axis=0)
    std = np.std(y, axis=0)

    # Normalize via the original dataset
    dataset.X = dataset.X - mean

    # Setup
    device = torch.device('cuda', args.gpu)

    # Set up architecture
    hidden = 128
    dropout = 0.3

    for trial in [3]:
        start_time = time.time()
        # Predictor
        predictor = nn.Sequential(
            nn.Linear(d_in + num_groups, hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden, d_out))

        # CMI Predictor
        value_network = nn.Sequential(
            nn.Linear(d_in + num_groups, hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden, num_groups))

        use_feature_costs = False
        if args.use_feature_costs:
            use_feature_costs = True

        mask_layer = MaskLayerGrouped(append=True, group_matrix=torch.tensor(feature_groups_mask))

        # Set up data loaders.
        train_dataloader = DataLoader(
            train_dataset, batch_size=128, shuffle=True, pin_memory=True,
            drop_last=True, num_workers=4)

        val_dataloader = DataLoader(
            val_dataset, batch_size=128, shuffle=False, pin_memory=True, drop_last=True, num_workers=4)

        test_dataloader = DataLoader(
            test_dataset, batch_size=128, shuffle=False, pin_memory=True, drop_last=True, num_workers=4)

        pretrain = MaskingPretrainer(
            predictor,
            mask_layer,
            lr=1e-3,
            loss_fn=nn.CrossEntropyLoss(),
            val_loss_fn=AUROC(task='multiclass', num_classes=3))

        trainer = Trainer(
            accelerator='gpu',
            devices=[args.gpu],
            max_epochs=200,
            num_sanity_val_steps=0
        )
        trainer.fit(pretrain, train_dataloader, val_dataloader)

        feature_costs = None

        if args.use_feature_costs:
            feature_cost_df = pd.read_csv("data/feature_list_intub-nw.csv")
            feature_costs = [feature_cost_df[feature_cost_df['Feature Name'] == feature]['Cost (Hours)'].item() for feature
                             in list(feature_groups_dict.keys())]
            print(feature_costs)
            feature_costs = np.array(feature_costs)

        # Jointly train value network and predictor
        print("Training CMI estimator")
        print("-"*8)
        run_description = f"max_features_40_with_decay_rate_0.2_use_entropy_false_feature_cost_{use_feature_costs}_trial_{trial}"
        logger = TensorBoardLogger("logs", name=f"{run_description}")

        checkpoint_callback = ModelCheckpoint(
                    save_top_k=1,
                    monitor='Perf Val/Final',
                    mode='max',
                    filename='best_val_perfomance_model',
                    verbose=False
                )

        checkpoint_callback_loss = ModelCheckpoint(
                    save_top_k=1,
                    monitor='Loss Val/Mean',
                    mode='min',
                    filename='best_val_loss_model',
                    verbose=False
                )

        greedy_cmi_estimator = CMIEstimator(value_network,
                                            predictor,
                                            mask_layer,
                                            lr=1e-3,
                                            min_lr=1e-6,
                                            max_features=40,
                                            eps=0.1,
                                            loss_fn=nn.CrossEntropyLoss(reduction='none'),
                                            val_loss_fn=AUROC(task='multiclass', num_classes=3),
                                            eps_decay=0.2,
                                            eps_steps=10,
                                            patience=5,
                                            feature_costs=feature_costs,
                                            cmi_scaling='positive')

        trainer = Trainer(accelerator='gpu',
                          devices=[args.gpu],
                          max_epochs=100,
                          precision=16,
                          logger=logger,
                          num_sanity_val_steps=0,
                          callbacks=[checkpoint_callback],
                          log_every_n_steps=10)

        trainer.fit(greedy_cmi_estimator, train_dataloader, val_dataloader)

        print("Training time=", time.time() - start_time)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name        | Type             | Params
--------------------------

Index(['faultNumber', 'VelocityRef:1', '<xdot>', '<BattSoc>', '<BattPwr>',
       '<Cltch1State>', '<Cltch2State>', '<BattV>', '<TransGear>', '<EngSpd>',
       '<IntkVlvLift>', '<EngTrq>', '<ThrPosPct>', '<WgAreaPct>',
       '<EgrVlvAreaPct>', '<VarCompRatioPos>', '<Acc>', '<Dec>', '<IgSw>',
       '<Chrg>', 'TransGear', 'BrkCmd', 'Cltch1Cmd', '<MotTrq>', '<StartTrq>',
       'StartCmd', 'MotTrqCmd', 'BattCrnt:1', 'MotPwrElec:1', 'MotPwrMech:1',
       'IntkVlvLiftCmd', 'FuelMainSoi', 'FuelFlw'],
      dtype='object')
Num groups= 32
Num features= 32
dict_keys(['VelocityRef:1', '<xdot>', '<BattSoc>', '<BattPwr>', '<Cltch1State>', '<Cltch2State>', '<BattV>', '<TransGear>', '<EngSpd>', '<IntkVlvLift>', '<EngTrq>', '<ThrPosPct>', '<WgAreaPct>', '<EgrVlvAreaPct>', '<VarCompRatioPos>', '<Acc>', '<Dec>', '<IgSw>', '<Chrg>', 'TransGear', 'BrkCmd', 'Cltch1Cmd', '<MotTrq>', '<StartTrq>', 'StartCmd', 'MotTrqCmd', 'BattCrnt:1', 'MotPwrElec:1', 'MotPwrMech:1', 'IntkVlvLiftCmd', 'FuelMainSoi', 'Fu

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152

Validation: 0it [00:00, ?it/s]

torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152

Validation: 0it [00:00, ?it/s]

torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152, 3]) torch.Size([1152])
torch.Size([1152